# <center>Basic Model for Supply Chain Transportation Lanes Using the Minimum Cost Flow Optimization Model </center>

<center>Sabi Horvat, January 2021</center>

This example model is similar to components of larger projects that I worked on from 2010-2014 with Xpress-Mosel.

What does this model do?  In summary, the model chooses transportation lanes from a number of options.  In practice, the data for these options is changing at least on a quarterly basis due to the following reasons.

* Contracts with the suppliers that manufacture these products expire and require periodic renegotiation. 
* Transportation costs from third party logistics (3PLs) fluctuate depending on fuel prices and other costs, but also due to shifting demand and supply.  One example is when a truck delivers products to an area where there is not also product to load upon a return trip.  When the 3PL can't coordinate for the truck to be highly utilized, such as in this situation, the truck is being paid to "deadhead" back empty or at other times at less than a Full TruckLoad (FTL).  These cost differences account for the increase in price for the quote you receive, compared to other destinations.

Additional python formulations of Minimum Cost Flow models can be found at: 
* [COIN-OR / PuLP (github) - American Steel Problem](https://github.com/coin-or/pulp/blob/master/examples/AmericanSteelProblem.py)
* [github-alerera - MCNF Timespace Problem](https://github.com/alerera/logistics-opt-python/blob/master/pulp/mcnf_timespace.py)
* [Gurobi-github - General Landing Page for Multiple Problems ](https://gurobi.github.io/modeling-examples/)

---
## Python Model

In [1]:
import pandas as pd
from pulp import *

## Data

In [2]:
# This data has been aggregated for period of time
# The transportation lane will be effective for this projected demand, during that time

# Data
#  Suppliers at Portland, Oregon and Portland, Maine
#  Distribution Center / Warehouse at Chicago, Illinois
#  Customer Demand in Seattle, Miama, Dallas, and San Diego

locations = ['Portland_OR','Portland_ME','Chicago_IL','Seattle_WA',
            'Miama_FL','Dallas_TX','SanDiego_CA']

supply_and_demand = {'Portland_OR':[100,0],
                     'Portland_ME':[100,0],
                     'Chicago_IL' :[0,50],
                     'Seattle_WA' :[0,40],
                     'Miama_FL'   :[0,30],
                     'Dallas_TX'  :[0,50],
                     'SanDiego_CA':[0,30]}
(supply, demand) = splitDict(supply_and_demand)

route_arcs = [   ('Portland_OR','Portland_OR'),
                 ('Portland_OR','Chicago_IL'),
                 ('Portland_OR','Seattle_WA'),
                 ('Portland_OR','Miama_FL'),
                 ('Portland_OR','Dallas_TX'),
                 ('Portland_OR','SanDiego_CA'),
                 ('Portland_ME','Portland_ME'),
                 ('Portland_ME','Chicago_IL'),
                 ('Portland_ME','Seattle_WA'),
                 ('Portland_ME','Miama_FL'),
                 ('Portland_ME','Dallas_TX'),
                 ('Portland_ME','SanDiego_CA'),
                 ('Chicago_IL','Seattle_WA'),
                 ('Chicago_IL','Miama_FL'),
                 ('Chicago_IL','Dallas_TX'),
                 ('Chicago_IL','SanDiego_CA')   ]

# cost per unit (variable per unit, fixed for utilizing lane)
route_cost = {   ('Portland_OR','Portland_OR'):[0,0],
                 ('Portland_OR','Chicago_IL'): [0.5,0],
                 ('Portland_OR','Seattle_WA'): [0.1,100],
                 ('Portland_OR','Miama_FL'):   [0.9,100],
                 ('Portland_OR','Dallas_TX'):  [0.5,100],
                 ('Portland_OR','SanDiego_CA'):[0.2,100],
                 ('Portland_ME','Portland_ME'):[0,0],
                 ('Portland_ME','Chicago_IL'): [0.4,100],
                 ('Portland_ME','Seattle_WA'): [0.9,100],
                 ('Portland_ME','Miama_FL'):   [0.3,100],
                 ('Portland_ME','Dallas_TX'):  [0.6,100],
                 ('Portland_ME','SanDiego_CA'):[0.9,100],
                 ('Chicago_IL','Seattle_WA'):  [0.5,0],
                 ('Chicago_IL','Miama_FL'):    [0.2,0],
                 ('Chicago_IL','Dallas_TX'):   [0.3,0],
                 ('Chicago_IL','SanDiego_CA'): [0.5,0]   }
(variable_costs, fixed_cost) = splitDict(route_cost)

## Model

In [3]:
# Decision Variables
flow = LpVariable.dicts("Route",route_arcs,lowBound=0,upBound=100,cat="Integer")

# Model, to minimize the objective function
model = LpProblem("Minimum_Cost_Flow_Problem_Sample",LpMinimize)

# Creates the objective function, product cost is not included in this simplified model
model += lpSum([flow[a]* variable_costs[a] for a in route_arcs]), "Transportation Cost "

# Constraint: Supply the demand
for l in locations:
    model += (supply[l]+ lpSum([flow[(i,j)] for (i,j) in route_arcs if j == l]) >=
             demand[l]+ lpSum([flow[(i,j)] for (i,j) in route_arcs if i == l])), "Flow %s"%l


## Results

In [4]:
# Solve the model
model.solve()

# The status of the solution (Optimal, Infeasible, Unbounded, Not Solved, or Undefined)
print("Status:", LpStatus[model.status])

# The objective solution
for v in model.variables():
    print(v.name, "=", v.varValue)

# The optimal objective function value 
print("Total Cost = ", value(model.objective))

Status: Optimal
Route_('Chicago_IL',_'Dallas_TX') = 0.0
Route_('Chicago_IL',_'Miama_FL') = 0.0
Route_('Chicago_IL',_'SanDiego_CA') = 0.0
Route_('Chicago_IL',_'Seattle_WA') = 0.0
Route_('Portland_ME',_'Chicago_IL') = 50.0
Route_('Portland_ME',_'Dallas_TX') = 20.0
Route_('Portland_ME',_'Miama_FL') = 30.0
Route_('Portland_ME',_'Portland_ME') = 0.0
Route_('Portland_ME',_'SanDiego_CA') = 0.0
Route_('Portland_ME',_'Seattle_WA') = 0.0
Route_('Portland_OR',_'Chicago_IL') = 0.0
Route_('Portland_OR',_'Dallas_TX') = 30.0
Route_('Portland_OR',_'Miama_FL') = 0.0
Route_('Portland_OR',_'Portland_OR') = 0.0
Route_('Portland_OR',_'SanDiego_CA') = 30.0
Route_('Portland_OR',_'Seattle_WA') = 40.0
Total Cost =  66.0
